In [1]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
import json
import requests
import gzip
import os

D:\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def read_jsonl_to_df(file_path, fields):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Parse each line as a JSON object
                json_obj = json.loads(line.strip())
                # Extract only the specified fields
                filtered_obj = {field: json_obj.get(field) for field in fields}
                data.append(filtered_obj)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue

    # Create a DataFrame from the list of filtered JSON objects
    df = pd.DataFrame(data)
    return df

In [5]:
# Fields to extract
fields = [
 'main_category', 'title', 'average_rating',
    'rating_number', 'features', 'description', 'price', 'images', 'videos',
    'store', 'categories', 'details', 'parent_asin', 'bought_together'
]

# Download and decompress the file
# decompressed_file_path = download_and_decompress(url, compressed_file_path)

# Read the dataset into a DataFrame
df  = read_jsonl_to_df('datasets/Beauty/meta_All_Beauty.jsonl/meta_All_Beauty.jsonl', fields)
df.to_csv('amazon_beauty_reviews.csv', index=False)


In [6]:
# Fields to extract
fields = [
    'rating', 'title', 'text', 'images', 'asin', 'parent_asin',
    'user_id', 'timestamp', 'verified_purchase', 'helpful_vote'
]

# Download and decompress the file
# decompressed_file_path = download_and_decompress(url, compressed_file_path)

# Read the dataset into a DataFrame
df  = read_jsonl_to_df('datasets/Beauty/All_Beauty.jsonl/All_Beauty.jsonl', fields)
df.to_csv('amazon_beauty_ratings.csv', index=False)

In [15]:
from CF_SVD import SVD_CF
from CF_item_based import ItemBasedCF
from dataset_processor import DatasetProcessor
from dataset_statistics import DatasetStatistics
import pandas as pd

from evaluator import EvaluateMetrics

ratings = pd.read_csv('processed_dataset/Amazon-Beauty/dataset_full_trimmed_beauty.csv')
trainset = pd.read_csv('processed_dataset/Amazon-Beauty/dataset_train_trimmed_beauty.csv')
testset = pd.read_csv('processed_dataset/Amazon-Beauty/dataset_test_trimmed_beauty.csv')

In [16]:
ratings

,rating,user_id,item_id
0,5.0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08P2DZB4X
1,5.0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B086QY6T7N
2,3.0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08DHTJ25J
3,5.0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07RBSLNFR
4,3.0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07SLFWZKN
...,...,...,...
35619,5.0,AGCESJO2YAHFW3GKOY2JSS5QC6DA,B07NXQYVMH
35620,5.0,AGCESJO2YAHFW3GKOY2JSS5QC6DA,B071RVQQBD
35621,5.0,AGNEYGHRA2MM5MUN6WCZRXDSNJZA,B005P4AIZ8
35622,2.0,AGNEYGHRA2MM5MUN6WCZRXDSNJZA,B005P49I7W


In [5]:
import surprise
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [17]:
# reader = Reader(rating_scale=(1, 5))
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

In [18]:
train_data = Dataset.load_from_df(trainset[['user_id', 'item_id', 'rating']], reader)
trainset = train_data.build_full_trainset()

In [19]:
test_data = Dataset.load_from_df(testset[['user_id', 'item_id', 'rating']], reader)
testing_set = test_data.build_full_trainset()

In [20]:
algo = SVD()

algo.fit(trainset)

In [22]:
temp = trainset.build_anti_testset()

MemoryError: 

In [ ]:
predictions = algo.test(testing_set.build_anti_testset())

In [3]:
svd = SVD_CF(train_df=trainset, full_df=data)
svd.fit()
svd.predict()
# predictions = svd.model.test(trainset.build_anti_testset())
recommendations = svd.generate_recommendations(k=5)
# print(recommendations)

In [12]:
isinstance(recommendations, pd.Series) or isinstance(recommendations, pd.DataFrame)


False

In [4]:
k=5
print('hi')
evaluator = EvaluateMetrics(testset)
results = evaluator.evaluate_recommendations(recommendations, k=5)
# print(f"NDCG@{k}: {results['NDCG@k']:.4f}")
# print(f"MRR@{k}: {results['MRR@k']:.4f}")
# print(f"HR@{k}: {results['HR@k']:.4f}")

hi
5
